In [1]:
import numpy as np

In [2]:
def get_ADC_samples_coordinates(gradient_path, dwelltime, nb_ADC_samples, ndim=2, verbose=0):
    """This function extracts the 2D and 3D sampling scheme from the gradient
        file, using the	dwelltime and the number of samples.

        Parameters
        ----------
        gradient_path: string
            Path to the gradient.txt file that contains the points of the gradient
        dwelltime: float
            dwell time extracted from the header of the acquisition, in nanoseconds
        nb_ADC_samples: int
            Number of points taken for this acquisition
        ndim: int
            Number of dimension of the gradient 2D or 3D acquisition
        verbose: int
            Verbosity level (default 0: silent)
        """
    # Load gardient file
    file_header = np.loadtxt(gradient_path, delimiter='\n', comments='\t')
    spokes_number = int(file_header[0])
    samples_per_spoke = int(file_header[1])
    file_content = np.loadtxt(gradient_path, delimiter='\t', skiprows=2,
                              ndmin=0)
    k0 = file_content[:spokes_number]
    grad = file_content[spokes_number:]
    
    # Over-sampling factor
    OS_factor = int(nb_ADC_samples*1.0 / (samples_per_spoke*spokes_number))
    real_samples_nb_per_spoke = OS_factor*samples_per_spoke
    
    
    # Compute gradient constants
    gradient_duration = samples_per_spoke * 0.01
    gamma = 42.576*1e6
    dt = 10e-6
    dt_ns = dt * 1e9  # Gradient time step in nanoseconds
    if verbose > 0:
        print('Number of Spokes                      -->', spokes_number)
        print('Number of samples / Spokes            -->', samples_per_spoke)
        print('Dimension                             -->', k0.shape[-1])
        print('Number of samples / Spokes in the ADC -->', real_samples_nb_per_spoke)
        print('Gradient duration was                 -->', gradient_duration)
        print('Gradient shape                        -->', grad.shape)
       
    
    grad = grad*1e-3  # Conversion from mT/m to T/m

    ## Start calcul of the ADC samples coordinates

    ADC_samples = []
    for k in range(spokes_number):
        gradient = grad[k*samples_per_spoke:(k+1)*samples_per_spoke, :]
        ADC_samples_k = np.zeros((real_samples_nb_per_spoke+1, ndim))
        ADC_samples_k[0] = k0[k]
        cnt = 1
        for j in range(1, nb_ADC_samples):
            ADC_time = dwelltime * j
            q = int(np.floor(ADC_time/dt_ns))
            r = ADC_time - (q)*dt_ns*1.0
            cnt = 1 + cnt
            if q < samples_per_spoke:
                gradient_to_sum = gradient[:q]
                ADC_samples_k[j] = ADC_samples_k[0] + (np.sum(gradient_to_sum,axis=0)*dt_ns + gradient[q,:]*r)*gamma*1e-9
            elif q==(samples_per_spoke) and (r==0):
                gradient_to_sum = gradient[:q, :]
                ADC_samples_k[j, :] = ADC_samples_k[0, :] + (np.sum(gradient_to_sum, axis=0)*dt_ns)*gamma*1e-9
            else:
                ADC_samples_k = ADC_samples_k[:cnt - 1, :]
                break
        ADC_samples.append(ADC_samples_k)

    return ADC_samples

In [3]:
dat_file = '/neurospin/lrmn/projets/MANIAC/20190513_invivo_3/jr140117/meas_MID00102_FID06273_ns_CSGRE_stack_N320_65x32x2049.dat'
import h5py
dwelltime=5000
dim = 3
if dim == 3:
    Gradient_filename='/neurospin/lrmn/projets/MANIAC/20190513_invivo_3/jr140117/GradientFile_stack_SPARKLING_N320_nc32x2049_OS1.txt'
    samples_matlab = h5py.File('/neurospin/lrmn/projets/MANIAC/20190513_invivo_3/jr140117/extracted_GradientFile_stack_SPARKLING_N320_nc32x2049_OS1.mat', 'r')['ADC_samples']
    nc = 2049
    osf = 2
    ns = 32
    nz = 65
    nb_ADC_samples = nc * ns * osf * nz
elif dim == 2:
    Gradient_filename='/neurospin/optimed/clazarus/DATA_CODE_28092018/Data/GradientFiles_2D/Sparkling_2D_MRM_revision_invivo/XP_N512_Tobs20ms_dt2us/sparkling/GradientFile_SPARKLING_N512_nc40x2001_OS5_dec35_tau1.txt'
    samples_matlab = h5py.File('/volatile/tmp/matlab.mat', 'r')['ADC_samples']
    nc = 2001
    osf = 5
    ns = 40
    nz = 1
    nb_ADC_samples = nc * ns * osf * nz
print(samples_matlab.shape)

(3, 8521760)


/home/le251198/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
samples = get_ADC_samples_coordinates(Gradient_filename, dwelltime, nb_ADC_samples, ndim=dim, verbose=1)
samples_ar = np.asarray(samples)
print(samples_ar.shape)
print(samples_ar.min())
print(samples_ar.max())

Number of Spokes ------------> 2080
Number of samples / Spokes --> 2048
(2080, 3)
Number of samples / Spokes --> 4096
Gradient duration was -------> 20.48
Gradient shape -------------> (4259840, 3)
(2080, 4097, 3)
-806.9067
807.5717


In [5]:
samples_ar = np.reshape(samples_ar, (np.prod(samples_ar.shape[:2]) , dim))

In [6]:
np.allclose(samples_ar.T, samples_matlab)

True

In [7]:
np.sum(np.abs(samples_ar.T - samples_matlab)**2)

0.0